In [1]:
# Global params 
from datetime import datetime

def nu_t():
    return datetime.utcnow().strftime("%H:%M:%S:%f")

my_detector = 'DS-20K'

In [2]:
from SNEWS_PT.snews_pub import CoincidenceTier, Publisher

pub = Publisher()
message = CoincidenceTier(detector_name=my_detector, nu_time=nu_t(), p_val = 0.98).message()
pub.send(message)

---------------------------------------------------------
_id                 :18_CoincidenceTier_21/11/15_19:07:39:662601
detector_name       :DS-20K
sent_time           :21/11/15 19:07:39
machine_time        :None
neutrino_time       :19:07:39:661173
p_value             :0.98


In [3]:
from SNEWS_PT.snews_pub import SignificanceTier, Publisher

pub = Publisher()
message = SignificanceTier(detector_name=my_detector, p_values=[0.78,0.67,0.75,0.56],nu_time = nu_t()).message()
pub.send(message)


---------------------------------------------------------
_id                 :18_SigTier_21/11/15_19:07:41:980653
detector_name       :DS-20K
sent_time           :21/11/15 19:07:41
machine_time        :None
neutrino_time       :19:07:41:979086
p_values            :[0.78, 0.67, 0.75, 0.56]


In [4]:
from SNEWS_PT.snews_pub import TimingTier, Publisher

pub = Publisher()
message = TimingTier(detector_name=my_detector, nu_time=nu_t(), timing_series = ['16:23:24"4546','16:23:24"4666','16:23:24"4756']).message()
pub.send(message)

---------------------------------------------------------
_id                 :18_TimeTier_21/11/15_19:07:43:307549
detector_name       :DS-20K
sent_time           :21/11/15 19:07:43
machine_time        :None
neutrino_time       :19:07:43:306589
timing_series       :['16:23:24"4546', '16:23:24"4666', '16:23:24"4756']


In [5]:
from SNEWS_PT.snews_pub import Retraction, Publisher

pub = Publisher()

message = Retraction(detector_name = my_detector, which_tier='CoincidenceTier', n_retract_latest=5, retraction_reason='someone spilled coffee on the DAQ').message()
pub.send(message)

AttributeError: 'Retraction' object has no attribute 'message_type'